In [1]:
import os
import math
import random
import pandas as pd
import numpy as np
import itertools

# 1. 数据集导入、处理

使用数据为[MovieLens-1M](https://grouplens.org/datasets/movielens/)

In [2]:
path = '../datasets/ml-1m'

`ratings.dat`格式为`user_id, movie_id, rating_score, timestamp`

- `user_id`介于$(1 - 6040)$
- `movie_id`介于$(1 - 3952)$
- 评分为5分制
- 每个用户有至少20个评分数据

In [3]:
rating_path = os.path.join(path, 'ratings.dat')

In [4]:
print('路径是否存在：', os.path.exists(rating_path), sep='')

路径是否存在：True


# 2. 思路

## 2.1 计算物品间的相似度

物品相似度：喜欢物品$i$的用户中同样喜欢物品$j$的个数。公式可进行如下定义（即是余弦相似度）：
$$
sim_{ij} = \frac{\vert N(i) \cup N(j)\vert}{\vert N(i)\vert \vert N(j)\vert}
$$


## 2.2 根据物品相似度和用户历史行为生成用户推荐列表

# 3. 算法实现

利用倒排表实现ItemBased CF

In [109]:
class ItemBasedCF:
    def __init__(self, path):
        self.train = {}
        self.test = {}
        self.N = {}
        self.S = {}
        self.generate_data(path)
        self.user_item_table = {} # 只统计正反馈物品
        
    def load_data(self, path):
        with open(path, 'rt', encoding='utf-8') as f:
            for line in f:
                yield line.strip('\n')
    
    def generate_data(self, path, random_pivot=0.8):
        # 建立了user-item表
        inx = 0
        print('{:=^20}'.format('前10行展示'))
        for line in self.load_data(path):
            user, movie, rating, _ = line.split('::')
            if inx < 10:
                print('No. {}: User_id: {}; Movie_id: {}; Rating: {}'.format(inx, user, movie, rating))
            inx += 1
            if random.random() < random_pivot:
                self.train.setdefault(user, {})
                self.train[user][movie] = int(rating)
            else:
                self.test.setdefault(user, {})
                self.test[user][movie] = int(rating)
            
    def compute_similarity(self):
        
        for user, items in self.train.items():
            for item, rating in items.items():
                # user_item倒排表中只统计正反馈物品，此处即评分>2分的电影
                self.user_item_table.setdefault(user, set())
                if rating > 2:
                    self.user_item_table[user].add(item)
                # 构造共现矩阵，不区分正反馈或负反馈物品，值存储在相似度矩阵中
                for item_v in items.keys():
                    if item != item_v:
                        self.S.setdefault(item, {})
                        self.S[item].setdefault(item_v, 0)
                        self.S[item][item_v] += 1
                # 构造物品被交互次数表
                self.N.setdefault(item, 0)
                self.N[item] += 1
        
        # 计算物品相似度矩阵，未出现在共现矩阵中的物品对无法计算相似度
        for item, items in self.S.items():
            for item_v in items.keys():
                self.S[item][item_v] /= math.sqrt(self.N[item] * self.N[item_v])
    
    def recommend(self, user, K=10, N=10):
        '''
        基于历史感兴趣电影的相似电影进行推荐
        '''
        if not self.S:
            print('先计算相似度矩阵！')
            return
        rank = {}
        positive_action_items = set(self.user_item_table[user])
        for item in positive_action_items:
            similar_items = sorted(self.S[item].items(), key=lambda x:x[1], reverse=True)[:K]
            for item_s, similar_degree in similar_items:
                rank.setdefault(item_s, 0)
                rank[item_s] += similar_degree * self.train[user][item]
        return sorted(rank.items(), key=lambda x:x[1], reverse=True)[:N]
    
    def evaluate(self):
        if not self.S:
            print('先计算相似度矩阵！')
            return
        hits, precision, recall = 0, 0, 0
        test_user_item = {}
        for user, items in self.test.items():
            for item, rating in items.items():
                if rating > 2:
                    test_user_item.setdefault(user, set())
                    test_user_item[user].add(item)
        for user, items in test_user_item.items():
            recommended = self.recommend(user)
            recommended = set(movie_simlar[0] for movie_simlar in recommended)
#             print(user, recommended)
#             print(test_user_item[user])
            hits += len(recommended & items)
            precision += len(recommended)
            recall += len(items)
        precision, recall = hits/precision, hits/recall
        return precision, recall

# 4. 模型测试

## 4.1 初始化

In [56]:
truncated_rating_path = os.path.join(path, 'small_ratings.dat')
with open(truncated_rating_path, 'w') as f1:
    with open(rating_path, 'r') as f2:
        f1.writelines(f2.readlines()[:1000])

In [110]:
item_based_cf = ItemBasedCF(rating_path)

=======前10行展示=======
No. 0: User_id: 1; Movie_id: 1193; Rating: 5
No. 1: User_id: 1; Movie_id: 661; Rating: 3
No. 2: User_id: 1; Movie_id: 914; Rating: 3
No. 3: User_id: 1; Movie_id: 3408; Rating: 4
No. 4: User_id: 1; Movie_id: 2355; Rating: 5
No. 5: User_id: 1; Movie_id: 1197; Rating: 3
No. 6: User_id: 1; Movie_id: 1287; Rating: 5
No. 7: User_id: 1; Movie_id: 2804; Rating: 5
No. 8: User_id: 1; Movie_id: 594; Rating: 4
No. 9: User_id: 1; Movie_id: 919; Rating: 4


## 4.2 用户推荐预测

In [111]:
item_based_cf.compute_similarity()

In [112]:
item_based_cf.recommend('9')

[('296', 43.4770726105737),
 ('608', 31.62903370269824),
 ('318', 31.37654623483121),
 ('593', 26.72635003808475),
 ('2858', 26.561778432639496),
 ('2571', 25.099805839400652),
 ('1265', 22.74408018292296),
 ('1704', 21.79015850644589),
 ('50', 21.249847202695264),
 ('2762', 18.211974606450067)]

In [113]:
item_based_cf.recommend('1')

[('1270', 21.282871656584316),
 ('594', 18.517609204033416),
 ('588', 18.469911497981975),
 ('595', 17.348695261085076),
 ('1196', 15.027640610651009),
 ('1022', 14.0858795193513),
 ('296', 13.184266028847524),
 ('364', 12.9332284575607),
 ('2716', 12.689139697118302),
 ('2858', 11.99878315342696)]

## 4.3 模型评估

In [115]:
precision, recall = item_based_cf.evaluate()

In [116]:
print('- 准确率：{:%}\n- 召回率：{:%}'.format(precision, recall))

- 准确率：12.745228%
- 召回率：4.589848%
